<a href="https://colab.research.google.com/github/juyeon-shin/CLUB-D.N.A./blob/main/2022/01.%20%ED%8E%AD%EA%B7%84%20%EB%AA%B8%EB%AC%B4%EA%B2%8C%20%EC%98%88%EC%B8%A1%20%EA%B2%BD%EC%A7%84%EB%8C%80%ED%9A%8C/03_%EB%AA%A8%EB%8D%B8%EB%A7%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np

In [3]:
train = pd.read_csv('/content/drive/MyDrive/2022/202201 펭귄 몸무게 예측 경진대회/데이터/pret_train.csv')
train

,id,Species,Island,Clutch Completion,Culmen Length (mm),Culmen Depth (mm),Flipper Length (mm),Sex,Delta 15 N (o/oo),Delta 13 C (o/oo),Body Mass (g)
0,0,Gentoo penguin (Pygoscelis papua),Biscoe,Yes,50.0,15.3,220,MALE,8.305150,-25.190170,5550
1,1,Chinstrap penguin (Pygoscelis antarctica),Dream,No,49.5,19.0,200,MALE,9.630740,-24.346840,3800
2,2,Gentoo penguin (Pygoscelis papua),Biscoe,Yes,45.1,14.4,210,FEMALE,8.519510,-27.018540,4400
3,3,Gentoo penguin (Pygoscelis papua),Biscoe,Yes,44.5,14.7,214,FEMALE,8.201060,-26.165240,4850
4,4,Gentoo penguin (Pygoscelis papua),Biscoe,No,49.6,16.0,225,MALE,8.383240,-26.842720,5700
...,...,...,...,...,...,...,...,...,...,...,...
106,109,Adelie Penguin (Pygoscelis adeliae),Torgersen,Yes,36.6,17.8,185,FEMALE,8.738543,-25.718804,3700
107,110,Adelie Penguin (Pygoscelis adeliae),Dream,Yes,39.2,18.6,190,MALE,9.110060,-25.795490,4250
108,111,Adelie Penguin (Pygoscelis adeliae),Dream,Yes,43.2,18.5,192,MALE,8.970250,-26.036790,4100
109,112,Chinstrap penguin (Pygoscelis antarctica),Dream,No,46.9,16.6,192,FEMALE,9.805890,-24.737350,2700


### 데이터 전처리
+ 범주형 변수 인코딩
------------------
train_test 분리
------------------
+ 정규화 (StandardScaler)
+ PCA

In [4]:
#범주형 변수 인코딩 -
#라벨인코딩은 회귀에는 적절하지 않음
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder()

#pandas
train = pd.get_dummies(train)
train

,id,Culmen Length (mm),Culmen Depth (mm),Flipper Length (mm),Delta 15 N (o/oo),Delta 13 C (o/oo),Body Mass (g),Species_Adelie Penguin (Pygoscelis adeliae),Species_Chinstrap penguin (Pygoscelis antarctica),Species_Gentoo penguin (Pygoscelis papua),Island_Biscoe,Island_Dream,Island_Torgersen,Clutch Completion_No,Clutch Completion_Yes,Sex_FEMALE,Sex_MALE
0,0,50.0,15.3,220,8.305150,-25.190170,5550,0,0,1,1,0,0,0,1,0,1
1,1,49.5,19.0,200,9.630740,-24.346840,3800,0,1,0,0,1,0,1,0,0,1
2,2,45.1,14.4,210,8.519510,-27.018540,4400,0,0,1,1,0,0,0,1,1,0
3,3,44.5,14.7,214,8.201060,-26.165240,4850,0,0,1,1,0,0,0,1,1,0
4,4,49.6,16.0,225,8.383240,-26.842720,5700,0,0,1,1,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,109,36.6,17.8,185,8.738543,-25.718804,3700,1,0,0,0,0,1,0,1,1,0
107,110,39.2,18.6,190,9.110060,-25.795490,4250,1,0,0,0,1,0,0,1,0,1
108,111,43.2,18.5,192,8.970250,-26.036790,4100,1,0,0,0,1,0,0,1,0,1
109,112,46.9,16.6,192,9.805890,-24.737350,2700,0,1,0,0,1,0,1,0,1,0


In [5]:
#독립변수, 타겟 데이터 지정

data = train.drop(['Body Mass (g)','id'],axis=1)
data = data.values
target = train['Body Mass (g)']

In [6]:
#훈련, 검증 데이터 분할

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data,target,test_size=0.1)

In [62]:
y_train_log = np.log(y_train)
y_test_log = np.log(y_test)

## 회귀

### 표준화 변환 - 일반선형회귀

In [42]:
#표준화 - PCA에 적합한 스케일링
#타겟을 제외한 모든 값을 표준 정규분포를 가지는 값으로 변환
#속성값의 영향력 없애기

from sklearn.preprocessing import StandardScaler
Scaler = StandardScaler()
Scaler.fit(X_train)
X_train_scaled = Scaler.transform(X_train)

In [43]:
# 일반선형회귀 모델 생성 & 학습
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(X_train_scaled,y_train)
print('절편 : ',lin_reg.intercept_)
print('회귀계수: ',lin_reg.coef_)

절편 :  4310.353535353535
회귀계수:  [ 117.05142935   81.27653086  185.29597809 -100.28057583   31.13850538
 -102.99311405 -181.11627727  253.71866573   26.93228602  -20.83360783
  -10.75950847    2.86075287   -2.86075287 -106.94704273  106.94704273]


In [44]:
# 일반선형회귀 모델 적용
X_test_scaled = Scaler.transform(X_test)

y_test_predict = lin_reg.predict(X_test_scaled)

In [45]:
# 오차 계산 RMSE
from sklearn.metrics import mean_squared_error
lin_mse = mean_squared_error(y_test,y_test_predict)
lin_rmse = np.sqrt(lin_mse)
print('RMSE ; ',lin_rmse)

RMSE ;  256.1276398944051


### 표준화 변환 - 일반선형회귀 - 타깃값 log 변환

In [63]:
#표준화 - PCA에 적합한 스케일링
#타겟을 제외한 모든 값을 표준 정규분포를 가지는 값으로 변환
#속성값의 영향력 없애기

from sklearn.preprocessing import StandardScaler
Scaler = StandardScaler()
Scaler.fit(X_train)
X_train_scaled = Scaler.transform(X_train)

In [64]:
# 일반선형회귀 모델 생성 & 학습
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(X_train_scaled,y_train_log)
print('절편 : ',lin_reg.intercept_)
print('회귀계수: ',lin_reg.coef_)

절편 :  8.35237550883329
회귀계수:  [ 0.01995629  0.02239068  0.03956037 -0.02831923  0.00561068 -0.02986619
 -0.03785635  0.06104351  0.00541816 -0.00388786 -0.00265859 -0.00037678
  0.00037678 -0.02607546  0.02607546]


In [65]:
# 일반선형회귀 모델 적용
X_test_scaled = Scaler.transform(X_test)

y_test_predict = lin_reg.predict(X_test_scaled)

In [66]:
# 오차 계산 RMSE
from sklearn.metrics import mean_squared_error
lin_mse = mean_squared_error(y_test_log,y_test_predict)
lin_rmse = np.sqrt(lin_mse)
print('RMSE ; ',lin_rmse)

RMSE ;  0.06471404455120211


### 표준화 변환 - 릿지회귀

In [50]:
#표준화 - PCA에 적합한 스케일링
#타겟을 제외한 모든 값을 표준 정규분포를 가지는 값으로 변환
#속성값의 영향력 없애기

from sklearn.preprocessing import StandardScaler
Scaler = StandardScaler()
Scaler.fit(X_train)
X_train_scaled = Scaler.transform(X_train)

In [51]:
# 릿지회귀 모델 생성 & 학습
from sklearn.linear_model import Ridge
rid_reg = Ridge()
rid_reg.fit(X_train_scaled,y_train)
print('절편 : ',rid_reg.intercept_)
print('회귀계수: ',rid_reg.coef_)

절편 :  4310.353535353535
회귀계수:  [ 115.47053589   68.83726081  188.24871804 -103.35681671   26.30942584
  -97.03643598 -168.43341292  237.15805325   31.41662178  -24.14914612
  -12.80071368    3.09731194   -3.09731194 -108.92482755  108.92482755]


In [52]:
# 릿지회귀 모델 적용
X_test_scaled = Scaler.transform(X_test)

y_test_predict = rid_reg.predict(X_test_scaled)

In [53]:
# 오차 계산 RMSE
from sklearn.metrics import mean_squared_error
rid_mse = mean_squared_error(y_test,y_test_predict)
rid_rmse = np.sqrt(rid_mse)
print('RMSE ; ',rid_rmse)

RMSE ;  257.6278182509209


### 표준화 변환 - 라쏘회귀

In [54]:
#표준화 - PCA에 적합한 스케일링
#타겟을 제외한 모든 값을 표준 정규분포를 가지는 값으로 변환
#속성값의 영향력 없애기

from sklearn.preprocessing import StandardScaler
Scaler = StandardScaler()
Scaler.fit(X_train)
X_train_scaled = Scaler.transform(X_train)

In [56]:
# 라쏘회귀 모델 생성 & 학습
from sklearn.linear_model import Lasso
las_reg = Lasso()
las_reg.fit(X_train_scaled,y_train)
print('절편 : ',las_reg.intercept_)
print('회귀계수: ',las_reg.coef_)

절편 :  4310.353535353535
회귀계수:  [ 1.13239588e+02  7.33329036e+01  1.86853340e+02 -9.97009950e+01
  2.62319601e+01 -0.00000000e+00 -8.33381259e+01  3.55293585e+02
  4.45477383e+01 -1.83348104e+00 -0.00000000e+00  4.73652487e+00
 -8.89972720e-15 -2.16357878e+02  0.00000000e+00]


In [57]:
# 라쏘회귀 모델 적용
X_test_scaled = Scaler.transform(X_test)

y_test_predict = las_reg.predict(X_test_scaled)

In [58]:
# 오차 계산 RMSE
from sklearn.metrics import mean_squared_error
rid_mse = mean_squared_error(y_test,y_test_predict)
rid_rmse = np.sqrt(rid_mse)
print('RMSE ; ',rid_rmse)

RMSE ;  257.4015120659024


### 표준화 변환 - PCA - 일반선형회귀

In [37]:
#표준화 - PCA에 적합한 스케일링
#타겟을 제외한 모든 값을 표준 정규분포를 가지는 값으로 변환
#속성값의 영향력 없애기

from sklearn.preprocessing import StandardScaler
Scaler = StandardScaler()
Scaler.fit(X_train)
X_train_scaled = Scaler.transform(X_train)

In [38]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 0.95)
pca.fit(X_train_scaled)
X_reduced = pca.transform(X_train_scaled)
print(X_reduced.shape)
print(pca.explained_variance_ratio_)

(99, 7)
[0.40066908 0.21924647 0.14908837 0.0989411  0.05565607 0.023117
 0.02040174]


In [39]:
# 일반선형회귀 모델 생성 & 학습
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(X_reduced,y_train)
print('절편 : ',lin_reg.intercept_)
print('회귀계수: ',lin_reg.coef_)

절편 :  4310.353535353535
회귀계수:  [-242.99221716 -170.91665838  172.9768835    27.53064057   14.01165944
  -47.34145666  121.35365692]


In [40]:
# 일반선형회귀 모델 적용
X_test_scaled = Scaler.transform(X_test)
X_test_reduced = pca.transform(X_test_scaled)

y_test_predict = lin_reg.predict(X_test_reduced)

In [41]:
# 오차 계산 RMSE
from sklearn.metrics import mean_squared_error
lin_mse = mean_squared_error(y_test,y_test_predict)
lin_rmse = np.sqrt(lin_mse)
print('RMSE ; ',lin_rmse)

RMSE ;  300.8644455474633


### 표준화 변환 - PCA - 일반선형회귀 - 타깃값 log 변환

In [68]:
#표준화 - PCA에 적합한 스케일링
#타겟을 제외한 모든 값을 표준 정규분포를 가지는 값으로 변환
#속성값의 영향력 없애기

from sklearn.preprocessing import StandardScaler
Scaler = StandardScaler()
Scaler.fit(X_train)
X_train_scaled = Scaler.transform(X_train)

In [69]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 0.95)
pca.fit(X_train_scaled)
X_reduced = pca.transform(X_train_scaled)
print(X_reduced.shape)
print(pca.explained_variance_ratio_)

(99, 7)
[0.40066908 0.21924647 0.14908837 0.0989411  0.05565607 0.023117
 0.02040174]


In [70]:
# 일반선형회귀 모델 생성 & 학습
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(X_reduced,y_train_log)
print('절편 : ',lin_reg.intercept_)
print('회귀계수: ',lin_reg.coef_)

절편 :  8.35237550883329
회귀계수:  [-0.05587813 -0.03861519  0.04012594  0.00518015  0.00381518 -0.0133948
  0.03131451]


In [71]:
# 일반선형회귀 모델 적용
X_test_scaled = Scaler.transform(X_test)
X_test_reduced = pca.transform(X_test_scaled)

y_test_predict = lin_reg.predict(X_test_reduced)

In [72]:
# 오차 계산 RMSE
from sklearn.metrics import mean_squared_error
lin_mse = mean_squared_error(y_test_log,y_test_predict)
lin_rmse = np.sqrt(lin_mse)
print('RMSE ; ',lin_rmse)

RMSE ;  0.07282617420639564


### 정규화 변환 - 일반선형회귀

In [46]:
# 정규화 변환
from sklearn.preprocessing import MinMaxScaler
Scaler = MinMaxScaler()
Scaler.fit(X_train)
X_train_scaled = Scaler.transform(X_train)

In [47]:
# 일반선형회귀 모델 생성 & 학습
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(X_train_scaled,y_train)
print('절편 : ',lin_reg.intercept_)
print('회귀계수: ',lin_reg.coef_)

절편 :  3591.95582692081
회귀계수:  [ 481.72727669  330.48790963  730.62071707 -423.2806083   104.20138
 -172.76184048 -384.73432763  557.49616811   61.9654617   -34.35830846
  -27.60715324    9.1028599    -9.1028599  -213.90499814  213.90499814]


In [48]:
# 일반선형회귀 모델 적용
X_test_scaled = Scaler.transform(X_test)

y_test_predict = lin_reg.predict(X_test_scaled)

In [49]:
# 오차 계산 RMSE
from sklearn.metrics import mean_squared_error
lin_mse = mean_squared_error(y_test,y_test_predict)
lin_rmse = np.sqrt(lin_mse)
print('RMSE ; ',lin_rmse)

RMSE ;  256.1276398944053


## 결정트리

### 스케일링x - 결정트리회귀

In [74]:
from sklearn.tree import DecisionTreeRegressor
dt_reg = DecisionTreeRegressor(max_depth = 5)
dt_reg.fit(X_train,y_train)
y_test_predict = dt_reg.predict(X_test)

array([4118.05555556, 4118.05555556, 5683.33333333, 5683.33333333,
       4118.05555556, 4118.05555556, 3775.        , 4891.66666667,
       4641.66666667, 3347.5       , 3552.5       , 5683.33333333])

In [75]:
# 오차 계산 RMSE
from sklearn.metrics import mean_squared_error
dt_mse = mean_squared_error(y_test,y_test_predict)
dt_rmse = np.sqrt(dt_mse)
print('RMSE ; ',dt_rmse)

RMSE ;  4577.4338180062305


### 스케일링x - 결정트리회귀 - 타겟 log 변환

In [76]:
from sklearn.tree import DecisionTreeRegressor
dt_reg = DecisionTreeRegressor(max_depth = 5)
dt_reg.fit(X_train,y_train_log)
y_test_predict = dt_reg.predict(X_test)

In [77]:
# 오차 계산 RMSE
from sklearn.metrics import mean_squared_error
dt_mse = mean_squared_error(y_test_log,y_test_predict)
dt_rmse = np.sqrt(dt_mse)
print('RMSE ; ',dt_rmse)

RMSE ;  0.08996911445409893


### 표준화 - 결정트리회귀

In [78]:
# 표준화
from sklearn.preprocessing import StandardScaler
Scaler = StandardScaler()
X_train_scaled = Scaler.fit_transform(X_train)

In [79]:
from sklearn.tree import DecisionTreeRegressor
dt_reg = DecisionTreeRegressor(max_depth = 5)
dt_reg.fit(X_train_scaled,y_train)

X_test_scaled = Scaler.transform(X_test)
y_test_predict = dt_reg.predict(X_test_scaled)

In [80]:
# 오차 계산 RMSE
from sklearn.metrics import mean_squared_error
dt_mse = mean_squared_error(y_test,y_test_predict)
dt_rmse = np.sqrt(dt_mse)
print('RMSE ; ',dt_rmse)

RMSE ;  250.1267939782613


### 표준화 - 결정트리회귀 - 타겟 log 변환

In [81]:
# 표준화
from sklearn.preprocessing import StandardScaler
Scaler = StandardScaler()
X_train_scaled = Scaler.fit_transform(X_train)

In [82]:
from sklearn.tree import DecisionTreeRegressor
dt_reg = DecisionTreeRegressor(max_depth = 5)
dt_reg.fit(X_train_scaled,y_train_log)

X_test_scaled = Scaler.transform(X_test)
y_test_predict = dt_reg.predict(X_test_scaled)

In [83]:
# 오차 계산 RMSE
from sklearn.metrics import mean_squared_error
dt_mse = mean_squared_error(y_test_log,y_test_predict)
dt_rmse = np.sqrt(dt_mse)
print('RMSE ; ',dt_rmse)

RMSE ;  0.08996911445409893
